In [1]:
"""──────────────────────────────────────────────────────────────────────────┐
│ Loading necessary libraries to build and train model                       │
└──────────────────────────────────────────────────────────────────────────"""
import os,sys,gc
import numpy as np
import pickle,glob
import torch
import proplot as plot
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

sys.path.insert(1, '/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/scikit/')
from tools import derive_var,read_and_proc
from tools.mlr import mlr
from tools.preprocess import do_eof,preproc_maria,preproc_haiyan
sys.path.insert(2, '../')
import read_stuff as read
%matplotlib inline
plot.rc.update({'figure.facecolor':'w','axes.labelweight':'ultralight',
                'tick.labelweight':'ultralight','gridminor.linestyle':'--','title.weight':'normal','linewidth':0.5})

/tmp/ipykernel_1715589/1556194838.py:19: ProplotWarning: rc setting 'linewidth' was renamed to 'meta.width' in version 0.8.
  plot.rc.update({'figure.facecolor':'w','axes.labelweight':'ultralight',


In [2]:
! pwd

/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/TCG_Rad_keras/maria


In [2]:
path = '/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/'
suffix = '_smooth_preproc_dict1b_g'
enter = '/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/pca/output/uvwheat/preproc2/'

maria_u = [read_and_proc.depickle(path+'TCGphy/2020_TC_CRF/dev/freddy0218/pca/output/uvwheat/preproc2/'+str(lime)+suffix)['u'] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
divider = np.asarray([maria_u[0][12:].shape[0],maria_u[1][25:].shape[0],maria_u[2][49:].shape[0],maria_u[3][85:].shape[0],maria_u[4][25:].shape[0]]).cumsum()

  0%|          | 0/5 [00:00<?, ?it/s]

In [3]:
folder = '/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/testML/output/maria/processed/timeseries/'#intermediate/'
Xtrain,Xvalid,Xtest,ytrain,yvalid,ytest = [],[],[],[],[],[]
for expname in [2,3,4]:
    obj = [read_and_proc.depickle(objs) for objs in sorted(glob.glob(folder+'inputoutput2/*val'+str(expname)))]
    Xtest.append(obj[0])
    Xtrain.append(obj[1])
    Xvalid.append(obj[2])
    ytest.append(obj[3])
    ytrain.append(obj[4])
    yvalid.append(obj[5])

TYPE = '3D'
if TYPE=='3D':
    folderpath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/testML/output/maria/processed/'
    pcastore = read_and_proc.depickle(folderpath+'PCA/PCAdict3D')
elif TYPE=='2D':
    folderpath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/testML/output/maria/processed/intermediate/'
    pcastore = read_and_proc.depickle(folderpath+'PCA/PCAdict')

/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/miniconda3/envs/fred_workenv/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator IncrementalPCA from version 1.1.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


# Models

In [4]:
class OptimMLR_lwsw_3D_ts_dropout(torch.nn.Module):
    def __init__(self,droprate):
        #super(OptimMLR_all_2D, self).__init__()
        super(OptimMLR_lwsw_3D_ts_dropout, self).__init__()
        ############################################################
        # Input channels
        ############################################################
        brchsize = [10,10]#[50,38,91,8,82,20,20]
        self.dense1 = torch.nn.Linear(brchsize[0], 1)
        self.dense2 = torch.nn.Linear(brchsize[1], 1)
        self.dropout1 = torch.nn.Dropout(droprate)
        self.dropout2 = torch.nn.Dropout(droprate)
        self.dropout3 = torch.nn.Dropout(droprate)
        ############################################################
        # Final Dense Layer
        ############################################################
        self.denseout = torch.nn.Linear(2,1)#106)
        
    def forward(self,X):
        brchindex = list(np.asarray([0,54,26,50,75,12,10,10]).cumsum())
        X_u, X_v, X_w, X_th = X[:,brchindex[0]:brchindex[1]],X[:,brchindex[1]:brchindex[2]],X[:,brchindex[2]:brchindex[3]],X[:,brchindex[3]:brchindex[4]]
        X_hdia, X_lw, X_sw = X[:,brchindex[4]:brchindex[5]],X[:,brchindex[5]:brchindex[6]],X[:,brchindex[6]:brchindex[7]]
        ############################################################
        # Optimal PC layer
        ############################################################
        X_lwc = self.dropout1(X_lw)
        bestlw = self.dense1(X_lwc)
        X_swc = self.dropout2(X_sw)
        bestsw = self.dense2(X_swc)
        ############################################################
        # Concat
        ############################################################
        bestPC = torch.cat((bestlw,bestsw),1)
        ############################################################
        # Prediction layer
        ############################################################
        bestPC = self.dropout3(bestPC)
        outpred = self.denseout(bestPC)
        return outpred
    
    def compute_l2_loss(self, w):
        return torch.square(w).sum()

In [5]:
class OptimMLR_lwsw_3D_ts(torch.nn.Module):
    def __init__(self,droprate):
        #super(OptimMLR_all_2D, self).__init__()
        super(OptimMLR_lwsw_3D_ts, self).__init__()
        ############################################################
        # Input channels
        ############################################################
        brchsize = [10,10]#[50,38,91,8,82,20,20]
        self.dense1 = torch.nn.Linear(brchsize[0], 1)
        self.dense2 = torch.nn.Linear(brchsize[1], 1)
        self.dropout1 = torch.nn.Dropout(droprate)
        self.dropout2 = torch.nn.Dropout(droprate)
        self.dropout3 = torch.nn.Dropout(droprate)
        ############################################################
        # Final Dense Layer
        ############################################################
        self.denseout = torch.nn.Linear(2,1)#106)
        
    def forward(self,X):
        brchindex = list(np.asarray([0,54,26,50,75,12,10,10]).cumsum())
        X_u, X_v, X_w, X_th = X[:,brchindex[0]:brchindex[1]],X[:,brchindex[1]:brchindex[2]],X[:,brchindex[2]:brchindex[3]],X[:,brchindex[3]:brchindex[4]]
        X_hdia, X_lw, X_sw = X[:,brchindex[4]:brchindex[5]],X[:,brchindex[5]:brchindex[6]],X[:,brchindex[6]:brchindex[7]]
        ############################################################
        # Optimal PC layer
        ############################################################
        #X_lwc = self.dropout1(X_lw)
        bestlw = self.dense1(X_lw)
        #X_swc = self.dropout2(X_sw)
        bestsw = self.dense2(X_sw)
        ############################################################
        # Concat
        ############################################################
        bestPC = torch.cat((bestlw,bestsw),1)
        ############################################################
        # Prediction layer
        ############################################################
        bestPC = self.dropout3(bestPC)
        outpred = self.denseout(bestPC)
        return outpred

    def compute_l2_loss(self, w):
        return torch.square(w).sum() 

In [6]:
import optuna
import ts_models

In [13]:
sys.path.insert(2, '../')
def get_bestparams(splitnum=None,n_trials=20,train='Yes'):
    X_totrain,y_totrain = read.train_optimizedMLR(folderpath,folderpath2).delete_padding(Xtrain[splitnum],ytrain[splitnum])#yall_orig[splitnum][23][0])
    X_tovalid,y_tovalid = read.train_optimizedMLR(folderpath,folderpath2).delete_padding(Xvalid[splitnum],yvalid[splitnum])#yall_orig[splitnum][23][1])   
    calc_device = 'cpu'
    ###################################################################################
    # Convert numpy arrays into tensors
    ###################################################################################
    train_Xtensor = torch.FloatTensor(X_totrain).to(calc_device)
    train_ytensor = torch.FloatTensor(y_totrain).to(calc_device)
    val_Xtensor = torch.FloatTensor(X_tovalid).to(calc_device)
    val_ytensor = torch.FloatTensor(y_tovalid).to(calc_device)
    train_data = torch.utils.data.TensorDataset(train_Xtensor, train_ytensor)
    val_data = torch.utils.data.TensorDataset(val_Xtensor, val_ytensor)
    batch_size = 5
    num_workers = 2
    train_loader = torch.utils.data.DataLoader(dataset=train_data,batch_size=batch_size,shuffle=True)
    val_loader = torch.utils.data.DataLoader(dataset=val_data,batch_size=batch_size,shuffle=False)

    def objective(trial):
        models,losses = [],[]
        droprate = trial.suggest_float("droprate",0.05,0.45)
        #model = OptimMLR_lwsw_3D_ts_dropout(droprate)
        model = OptimMLR_lwsw_3D_ts(droprate)
        lr = trial.suggest_float("lr",1e-5,1e-3)#,log=True)
        optimizer = torch.optim.Adam(model.parameters(),lr=lr)
        criterion = torch.nn.L1Loss()
        n_epochs = 1500
        scheduler2 = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=1e-8, max_lr=1e-5,cycle_momentum=False)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',min_lr=1e-12)
        l2_lambda = trial.suggest_float("l2_lambda",0.01,0.02)

        train_losses = []
        val_losses = []
        for epoch in range(1,n_epochs+1):
            loss = 0
            for features, labels in train_loader:
                optimizer.zero_grad() # Clears existing gradients from previous epoch
                output = model(features)
                batch_loss = criterion(output, labels.unsqueeze(1))
                batch_loss.backward()
                optimizer.step()
            loss += batch_loss.item()
            loss = loss/len(train_loader)
            train_losses.append(loss)
            val_loss = ts_models.eval_model(model,
                                            val_loader,
                                            criterion,
                                            l2_lambda)
            val_losses.append(val_loss)
            if epoch%100 == 0:
                print('Epoch: {}/{}.............'.format(epoch, n_epochs))
                print("Loss: {:.4f}".format(loss))
        return loss
    if train=='Yes':
        study = optuna.create_study(directions=["minimize"])
        study.optimize(objective, n_trials=n_trials)#, timeout=300)
        return study, train_loader, val_loader
    else:
        return train_loader, val_loader

In [22]:
folderpath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/TCG_Rad_keras/store/'
folderpath2='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/testML/output/haiyan/processed/new3D/'
#bestparams,train_loader,val_loader = get_bestparams(splitnum=2,n_trials=20)
#read_and_proc.save_to_pickle('./models/1d/nomc2/4/bestparams.pkt',bestparams,'PICKLE')
bestparams = read_and_proc.depickle('./models/1d/mcdrop2/4/bestparams.pkt')
train_loader,val_loader = get_bestparams(splitnum=2,n_trials=20,train='No')

#### 0 best params

In [64]:
bestparams.best_params

{'droprate': 0.34831132482125615,
 'lr': 0.0007215452041872332,
 'l2_lambda': 0.017553586437368145}

#### train

In [23]:
times = ['exp1a','exp1b','exp1c','exp1d','exp1e','exp1f','exp1g']
#times = ['exp2a','exp2b','exp2c']#,'exp1d','exp1e']
exp = 4
for i in tqdm(range(len(times))):
    models,losses = [],[]
    #model = OptimMLR_lwsw_3D_ts(bestparams.best_params['droprate'])
    model = OptimMLR_lwsw_3D_ts_dropout(bestparams.best_params['droprate'])
    optimizers = [torch.optim.Adam(model.parameters(), lr=bestparams.best_params['lr'])]#, optim.AdaBound(model.parameters(),lr=1e-7)] 1e-6
    loss = torch.nn.MSELoss()
    for optimizer in optimizers:
        scheduler2 = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=1e-9, max_lr=2e-5,cycle_momentum=False)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',min_lr=1e-18)
        num_epochs = 1000*22#26
        early_stopper = ts_models.EarlyStopping(patience=260, verbose=False, delta=1e-8, path='checkpoint.pt', trace_func=print)#EarlyStopper(patience=8, min_delta=1e-3)
                #variance_store = [varu,varv,varw,varth]
                #variance_store = [varu,varv,varth]
        model,loss = ts_models.train_model(model=model,optimizer=optimizer,scheduler=[scheduler,scheduler2],numepochs=num_epochs,early_stopper=early_stopper,variance_store=None,\
                                 lossfunc=loss,train_loader=train_loader,val_loader=val_loader,test_loader=None,l2_lambda=bestparams.best_params['l2_lambda'])
        models.append(model)
        losses.append(loss)
    torch.save(models, './models/1d/mcdrop2/'+str(exp)+'/models_LWSW1d_1115_'+str(times[i])+'.pt')
    read_and_proc.save_to_pickle('./models/1d/mcdrop2/'+str(exp)+'/losses_LWSW1d_1115_'+str(times[i])+'.pkt',losses,'PICKLE')

  0%|          | 0/7 [00:00<?, ?it/s]

(0.32199541411616583, 0.21542634842917324)
(0.10165733098983765, 0.09722645892761647)
(0.02428903392600742, 0.05143971294164658)
(0.008395023538138379, 0.027051787870004774)
(0.0037905643528740093, 0.016580600757151842)
(0.00203462476751652, 0.010793128306977451)
(0.0010104650996667756, 0.00801533421035856)
(0.0007366331986469794, 0.0067322135902941225)
(0.0003509114759584587, 0.005304471380077303)
(0.00021590680241636372, 0.004275644407607615)
(9.456359330008787e-05, 0.0035904138116165996)
(1.774300739818748e-05, 0.0030034224735572936)
(1.7040907229759483e-07, 0.0026080514187924566)
(6.777369941772401e-09, 0.002575498353689909)
(5.595357714408253e-09, 0.002556220896076411)
(4.7304245230513206e-09, 0.0025492351618595422)
(4.308894545303577e-09, 0.002538931043818593)
(4.319489157769948e-09, 0.002526903385296464)
(3.327782683884814e-09, 0.0025135244242846964)
(3.701734616120354e-09, 0.00249289731727913)
(2.2383785655176835e-09, 0.002475978271104395)
(3.3668009075116245e-09, 0.00245935365